In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
results_df = pd.read_csv('F1KaggleData/results.csv')
races_df = pd.read_csv('F1KaggleData/races.csv')
drivers_df = pd.read_csv('F1KaggleData/drivers.csv')
laptimes_df = pd.read_csv('F1KaggleData/laptimes.csv')


In [ ]:
display(results_df.sample(20))

In [ ]:
# Find the fastest lap for a given race. 

def enrich_lap_times(races_df, laptimes_df, drivers_df):

    races_narrow_df = races_df[['raceId', 'year', 'name']]
    drivers_narrow_df = drivers_df[['driverId','driverRef']]

    enriched_laptimes_df = pd.merge(laptimes_df, races_narrow_df, on='raceId')
    return pd.merge(enriched_laptimes_df, drivers_narrow_df, on='driverId')


enriched_laptimes_df = enrich_lap_times(races_df, laptimes_df, drivers_df)

display(enriched_laptimes_df.sample(10))

In [ ]:
fastest_laps = enriched_laptimes_df.loc[enriched_laptimes_df.groupby(['year', 'name'])['milliseconds'].idxmin()]

pd.set_option('display.max_rows', 100)

fastest_laps_minimal = fastest_laps[['year','name','driverRef', 'milliseconds']]

# display(fastest_laps_minimal)
display(fastest_laps_minimal.loc[fastest_laps_minimal['year'] == 1996])
# display(fastest_laps_minimal)

In [ ]:
import copy

multi_index_fast_laps = copy.copy(fastest_laps_minimal).set_index(['year', 'name'])


In [ ]:
display(multi_index_fast_laps.loc[1998])

In [ ]:
from ipywidgets import interact, interact_manual

@interact
def display_fastest_laps_for_year(year=(1996, 2014, 1)):
    
    if year not in multi_index_fast_laps.index:
        return 'No fastest laps data available for: {}'.format(year)
    else:
        fastest_laps_for_year = multi_index_fast_laps.loc[year]['driverRef'].value_counts().to_frame()
        fastest_laps_for_year.rename(columns = {'driverRef':'FastestLaps'}, inplace = True)
        return fastest_laps_for_year
    
    


In [ ]:
display(display_fastest_laps_for_year(1998))

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



In [ ]:
# display(multi_index_fast_laps.sample(20))
def get_fastest_laps_for_circuit(name):

    fastest_laps_for_circuit = copy.copy(multi_index_fast_laps.loc[pd.IndexSlice[:, name], :])
    fastest_laps_for_circuit['time'] = pd.to_timedelta(fastest_laps_for_circuit['milliseconds'], unit='ms')
    
    return fastest_laps_for_circuit
    
    
circuits = multi_index_fast_laps.index.get_level_values('name').unique().to_list()

interact(get_fastest_laps_for_circuit, name=circuits)

In [ ]:
def graph_fastest_laps(name):
    
    fastest_laps_for_circuit = get_fastest_laps_for_circuit(name)
    fastest_laps_for_circuit.drop(['driverRef', 'milliseconds'], axis=1, inplace=True)
    fastest_laps_for_circuit.reset_index(level='name', inplace=True, drop=True)
    
    fastest_laps_for_circuit.plot()
    
    display(fastest_laps_for_circuit)
    
# graph_fastest_laps('Monaco Grand Prix')


interact(graph_fastest_laps, name=circuits)

ideas

Dominance, teams of Mercedes, REdBull, McLaren etc and show wins per season over the last 20y

driver wins, fastest laps, coloured byteam

show non top 3 team podiums -. ie. basically none

downfall of Williams

mid table... see how that seems more competitive

back marker teams... who picked up what points... 

points allocated to the non Top3 teams... 